In [2]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split
from torch.cuda.amp import autocast, GradScaler


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
# %cd /content/drive/MyDrive/
# !kaggle datasets download -d ashery/chexpert -p chexpert_data

In [29]:
model = timm.create_model('vit_base_patch16_224', pretrained=True)

In [30]:
class CheXpertDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):

        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # مسیر نسبی تصویر از ستون CSV
        img_rel_path = self.labels_df.iloc[idx]['Path']

        # حذف پیشوند "CheXpert-v1.0-small" اگر در مسیر بود
        if img_rel_path.startswith("CheXpert-v1.0-small"):
            img_rel_path = img_rel_path[len("CheXpert-v1.0-small")+1:]  # +1 برای حذف اسلش بعدی

        # مسیر کامل تصویر با join کردن مسیر ریشه و مسیر نسبی اصلاح شده
        img_path = os.path.join(self.img_dir, img_rel_path)

        # بارگذاری تصویر با تبدیل به RGB (3 کاناله)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"[WARNING] Could not load image: {img_path} -- {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        if self.transform:
            image = self.transform(image)

        # گرفتن لیبل‌ها و infer_objects برای جلوگیری از warning
        labels = self.labels_df.iloc[idx][['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']]
        labels = labels.infer_objects(copy=False).fillna(0).values.astype('float32')

        return image, labels



In [31]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [32]:
base_csv_file='/content/drive/MyDrive/chexpert_data_v2/train.csv'
df = pd.read_csv(base_csv_file)


In [33]:
base_valid_file='/content/drive/MyDrive/chexpert_data_v2/valid.csv'
dfvalid = pd.read_csv(base_valid_file)


In [34]:
df_subset = df.sample(frac=0.3, random_state=42).reset_index(drop=True)
df_valid_subset=dfvalid.sample(frac=0.3, random_state=42).reset_index(drop=True)

In [35]:
df_subset.to_csv("chexpert_30percent.csv", index=False)


In [36]:
df_valid_subset.to_csv("chexpert_30percen_valid.csv", index=False)

In [37]:
train_dataset = CheXpertDataset(
    csv_file="chexpert_30percent.csv",
    img_dir='/content/drive/MyDrive/chexpert_data_v2/',
    transform=transform
)

In [38]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)


In [39]:
val_dataset = CheXpertDataset(
    csv_file="chexpert_30percen_valid.csv",
    img_dir='/content/drive/MyDrive/chexpert_data_v2/',
    transform=transform
)

In [40]:
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [41]:
model = model.to(device)

In [42]:
model.head = nn.Linear(model.head.in_features, 5)
model = model.to(device)

In [43]:
criterion = nn.BCEWithLogitsLoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [45]:
import gc

del images, labels, outputs, loss
gc.collect()
torch.cuda.empty_cache()


In [1]:
scaler = GradScaler()
num_epochs = 5
val_subset_ratio = 0.3  # فقط 30٪ از val برای ارزیابی

for epoch in range(num_epochs):
    print(f"\n📚 Epoch {epoch+1}/{num_epochs}")

    # -------------------------
    # 🔁 مرحله Training
    # -------------------------
    model.train()
    train_loss = 0.0
    train_loop = tqdm(train_loader, desc="🔧 Training", leave=False)

    for images, labels in train_loop:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).float()

        optimizer.zero_grad()
        with autocast():  # mixed precision training
            outputs = model(images)
            mask = (labels != -1).float()
            loss_raw = criterion(outputs, labels)
            loss = (loss_raw * mask).sum() / mask.sum()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item() * images.size(0)
        train_loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader.dataset)
    print(f"✅ Avg Train Loss: {avg_train_loss:.4f}")

    # -------------------------
    # 🔍 مرحله Validation
    # -------------------------
    model.eval()
    val_loss = 0.0
    val_loop = tqdm(val_loader, desc="🧪 Validating", leave=False)
    max_val_batches = int(len(val_loader) * val_subset_ratio)

    with torch.no_grad():
        for i, (images, labels) in enumerate(val_loop):
            if i > max_val_batches:
                break

            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True).float()

            with autocast():
                outputs = model(images)
                mask = (labels != -1).float()
                loss_raw = criterion(outputs, labels)
                loss = (loss_raw * mask).sum() / mask.sum()

            val_loss += loss.item() * images.size(0)
            val_loop.set_postfix(loss=loss.item())

    avg_val_loss = val_loss / (max_val_batches * val_loader.batch_size)
    print(f"🧪 Avg Val Loss: {avg_val_loss:.4f}")

NameError: name 'GradScaler' is not defined

In [20]:
import os
print(os.path.exists('/content/drive/MyDrive/chexpert_data_v2/train/patient20948/study2/view1_frontal.jpg'))


True


✅ Image is valid.
